### Import Python Libraries

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import cross_validation
from sklearn.svm import SVC
from sklearn.naive_bayes import  MultinomialNB
from sklearn.metrics import precision_recall_fscore_support
import numpy as np
import codecs
import os
import sys
import time
from sklearn.feature_selection import SelectPercentile
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.cross_validation import StratifiedKFold
from pandas import DataFrame
import numpy
import csv
import pyap
 
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp

from sklearn.preprocessing import StandardScaler
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import KFold
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression as LogR
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


### Create function for Stratified K-Fold

In [2]:
def stratified_k_fold():
    from sklearn.cross_validation import StratifiedKFold
    from pandas import DataFrame
    import numpy
    data = DataFrame(df,columns=names)
    content_data = data['Content'].values

    y = data['Classifier'].values
 

    k_fold = StratifiedKFold(y, n_folds=10, shuffle=False, random_state=None)
    scores = []
    for train_indices, test_indices in k_fold:
        X_train = [content_data[li] for li in train_indices]
        Y_train = [y[li] for li in train_indices]
        X_test = [content_data[li] for li in test_indices]
        Y_test = [y[li] for li in test_indices]
   
        pipeline.fit(X_train, Y_train)
        score = pipeline.score(X_test, Y_test)
        scores.append(score)
        predictions = pipeline.predict(X_test)
        print(confusion_matrix(Y_test, predictions))
        
    score = sum(scores) / len(scores)
    print("StratifiedKFold Cross Validation Score - Using 10 folds")
    print(score) 

### Create ROC Curve

In [3]:
def ROC_Curve():    
    data = DataFrame(df,columns=names)
    content_data = data['Content'].values
 
    y = data['Classifier'].values

    from scipy import interp
    from sklearn.metrics import roc_curve, auc
    k_fold = StratifiedKFold(y, n_folds=10, shuffle=False, random_state=None)
    scores = []
    mean_tpr = 0.0
    mean_fpr = np.linspace(0, 1, 100)
    all_tpr = []

    for i,(train_indices, test_indices) in enumerate(k_fold):
        X_train = [content_data[li] for li in train_indices]
        Y_train = [y[li] for li in train_indices]
        X_test = [content_data[li] for li in test_indices]
        Y_test = [y[li] for li in test_indices]
 
        pipeline.fit(X_train, Y_train)
        y_pred_rf = pipeline.predict_proba(X_test)[:, 1]
  

        #fpr_rf, tpr_rf, _ = roc_curve(Y_test[1:], y_pred_rf[1:],pos_label='0')
    
        # Compute ROC curve and area the curve
        fpr, tpr = roc_curve(Y_test[1:], y_pred_rf[1:],pos_label='0')
        mean_tpr += interp(mean_fpr, fpr, tpr)
        mean_tpr[0] = 0.0
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, lw=1, label='ROC fold %d (area = %0.2f)' % (i, roc_auc))
        score = pipeline.score(X_test, Y_test)
        scores.append(score)
        predictions = pipeline.predict(X_test)
        #plt.figure(1)
        #plt.plot([0, 1], [0, 1], 'k--')
        #plt.plot(fpr_rf, tpr_rf, label='RF')
        #plt.xlabel('False positive rate')
        #plt.ylabel('True positive rate')
        #plt.title('ROC curve')
        #plt.legend(loc='best')
        #plt.show()

    mean_tpr /= len(k_fold)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    plt.plot(mean_fpr, mean_tpr, 'k--',
             label='Mean ROC (area = %0.2f)' % mean_auc, lw=2)

    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()


### Upload csv file into dataframe

In [4]:
names=['URL','Classifier','Content']
df = pd.read_csv('URLCONTENT_NEW3_Results2.csv',sep=',', names=["URL","Classifier","Content"])
print("Successfully read the csv file")
print(len(df.index))

Successfully read the csv file
607


### Random split of dataset (80% training data and 20% test data)

In [5]:
#split the data into training and testing sets. Default is 80% train.
#After the split, we have two arrays of arrays
train,test = cross_validation.train_test_split(df,train_size=0.80)
train_data,test_data = pd.DataFrame(train,columns=names),pd.DataFrame(test,columns=names)
print("Successfully split the dataset into test and training dataset")
print (len(train_data.index))
print(len(test_data.index))
print("Train Data Report")
train_data_count = train_data.groupby('Classifier')
print(train_data_count.count())
print("Test Data Report")
test_data_count = test_data.groupby('Classifier')
print(test_data_count.count())

Successfully split the dataset into test and training dataset
485
122
Train Data Report
            URL  Content
Classifier              
0           282      282
1           203      203
Test Data Report
            URL  Content
Classifier              
0            68       68
1            54       54


In [7]:
def display_scores(vectorizer,tfidf_result):
    scores = zip(vectorizer.get_feature_names(),np.asarray(tfidf_result.sum(axis=0)).ravel())
    sorted_scores = sorted(scores,key=lambda x:x[1],reverse=True)
    #for item in sorted_scores:
     #   print "{0:50} Score: {1}".format(item[0],item[1])

### Create features using TF-IDF Vectorizer

In [8]:
#Vectorization is the process of converting all text into a binary vector
vectorizer = TfidfVectorizer(norm='l2', min_df=3,  #discard words appearing in less than 3 rows
                             max_df = 0.8,#discard words appering in more than 80% of the rows
                             sublinear_tf=True,
                             use_idf=True, strip_accents='unicode',  # replace all accented unicode char,
                             token_pattern=r'\w{4,}',  # tokenize only words of 4+ chars
                             ngram_range=(1, 2),  # features made of a single tokens
                             smooth_idf=False,  # prevents zero division for unseen words
                             decode_error='ignore',  analyzer='word', lowercase=True, stop_words='english')

#fit on training data
#fit_transform() will create the vocabulary (i.e. the list of words/features) and the feature weights from the training data. 
x_train_matrix = vectorizer.fit_transform(train_data['Content'].values)
#print(x_train_matrix.toarray())
# print the number of rows in training dataset and the number of features
#print(x_train_matrix.shape)
display_scores(vectorizer,x_train_matrix)
y_train_matrix = train_data['Classifier'].values

y_test_matrix = test_data['Classifier'].values

#call the transform() on the test data, which will create the feature weights for the test data, using the same vocabulary as the training data.
x_test_matrix= vectorizer.transform(test_data['Content'].values)
print("Train Features List")
#pd.DataFrame(x_train_matrix.toarray())
# print the number of rows in training dataset and the number of features
#print(x_test_matrix.shape)
#print(x_test_matrix)



Train Features List


### Display top features in order

In [9]:
print ("Top Feature Names")
import codecs
import sys

def display_scores(vectorizer, tfidf_result):
    scores = zip(vectorizer.get_feature_names(),
                 np.asarray(tfidf_result.sum(axis=0)).ravel())
    sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)
    for item in sorted_scores:
        print "{0:50} Score: {1}".format(item[0], item[1])

def main():
    tfidf_result = x_train_matrix
    display_scores(vectorizer, tfidf_result)

if __name__ == '__main__':
    sys.stdout = codecs.getwriter('utf-8')(sys.stdout)
    main()
print ("End of Feature Names")

Top Feature Names
food                                               Score: 18.469513837
pantry                                             Score: 16.4074522688
food pantry                                        Score: 15.1123134169
community                                          Score: 10.2618153321
services                                           Score: 9.50248813778
contact                                            Score: 9.46029529338
help                                               Score: 9.25422162303
need                                               Score: 8.55220407403
home                                               Score: 8.51059935549
church                                             Score: 7.89868451175
information                                        Score: 7.88006600658
donate                                             Score: 7.84950035523
volunteer                                          Score: 7.63656384147
employment                                     

### Show top features by n-gram range

In [10]:
#print top features by n-gram
from collections import defaultdict
features_by_gram = defaultdict(list)
for f, w in zip(vectorizer.get_feature_names(), vectorizer.idf_):
    features_by_gram[len(f.split(' '))].append((f, w))
top_n = 2
print "Top Features by n-gram(bi-gram)"
for gram, features in features_by_gram.iteritems():
    top_features = sorted(features, key=lambda x: x[1], reverse=True)[:top_n]
    top_features = [f[0] for f in top_features]
    print '{}-gram top:'.format(gram), top_features

Top Features by n-gram(bi-gram)
1-gram top: [u'0000notification', u'10001130']
2-gram top: [u'1000 1130', u'1000 noon']


### Mulitnomial NB Cross Validation Module

In [19]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from collections import deque
count_vectorizer = CountVectorizer()
pipeline = Pipeline([
  ('count_vectorizer',   CountVectorizer(ngram_range=(1, 2))),
  ('tfidf_transformer',  TfidfTransformer()),
  ('classifier',         MultinomialNB()) ])
stratified_k_fold()
#ROC_Curve()

from sklearn.externals import joblib
joblib.dump(pipeline, "Classifier1.pkl", compress=9)

[[34  1]
 [ 1 25]]
[[35  0]
 [ 0 26]]
[[34  1]
 [ 3 23]]
[[35  0]
 [ 0 26]]
[[35  0]
 [ 0 26]]
[[35  0]
 [ 0 26]]
[[35  0]
 [23  3]]
[[35  0]
 [20  5]]
[[35  0]
 [18  7]]
[[35  0]
 [20  5]]
StratifiedKFold Cross Validation Score - Using 10 folds
0.855792349727


['Classifier1.pkl']

### Bernouli NB Cross Validation Module

In [20]:
from sklearn.naive_bayes import BernoulliNB
 
pipeline = Pipeline([
  ('count_vectorizer',   CountVectorizer(ngram_range=(1, 2))),
  ('classifier',         BernoulliNB(binarize=0.0)) ])
stratified_k_fold()
#ROC_Curve()

from sklearn.externals import joblib
joblib.dump(pipeline, "Classifier2.pkl", compress=9)

[[35  0]
 [24  2]]
[[34  1]
 [22  4]]
[[35  0]
 [22  4]]
[[35  0]
 [24  2]]
[[35  0]
 [23  3]]
[[35  0]
 [20  6]]
[[35  0]
 [25  1]]
[[35  0]
 [24  1]]
[[35  0]
 [25  0]]
[[35  0]
 [23  2]]
StratifiedKFold Cross Validation Score - Using 10 folds
0.61606557377


['Classifier2.pkl']

### SVM, kernel=linear Cross Validation Module

In [21]:
 # Perform classification with SVM, kernel=linear
from sklearn.linear_model import SGDClassifier
pipeline = Pipeline([
  ('count_vectorizer',   CountVectorizer(ngram_range=(1, 2))),
  ('classifier', SGDClassifier())])
stratified_k_fold()
#ROC_Curve()
    
from sklearn.externals import joblib
joblib.dump(pipeline, "Classifier3.pkl", compress=9)

[[29  6]
 [ 0 26]]
[[31  4]
 [ 0 26]]
[[30  5]
 [ 1 25]]
[[28  7]
 [ 0 26]]
[[34  1]
 [ 0 26]]
[[34  1]
 [ 0 26]]
[[35  0]
 [15 11]]
[[33  2]
 [16  9]]
[[32  3]
 [ 5 20]]
[[34  1]
 [ 8 17]]
StratifiedKFold Cross Validation Score - Using 10 folds
0.876092896175


['Classifier3.pkl']

### SVM - LinearSVC Cross Validation Module

In [22]:
 # Perform classification with SVM - LinearSVC
pipeline = Pipeline([
  ('count_vectorizer',   CountVectorizer(ngram_range=(1, 2))),
  ('classifier', svm.LinearSVC())])
stratified_k_fold()
#ROC_Curve()

from sklearn.externals import joblib
joblib.dump(pipeline, "Classifier4.pkl", compress=9)

[[26  9]
 [ 2 24]]
[[32  3]
 [ 0 26]]
[[32  3]
 [ 0 26]]
[[33  2]
 [ 1 25]]
[[32  3]
 [ 0 26]]
[[35  0]
 [ 0 26]]
[[33  2]
 [18  8]]
[[35  0]
 [16  9]]
[[34  1]
 [ 4 21]]
[[33  2]
 [ 9 16]]
StratifiedKFold Cross Validation Score - Using 10 folds
0.876174863388


['Classifier4.pkl']

### Logistic Regression Cross Validation Module

In [23]:
#Perform classification with Logistic Regression
from sklearn import metrics
pipeline = Pipeline([
  ('count_vectorizer',   CountVectorizer(ngram_range=(1, 2))),
  ('classifier', LogisticRegression(C=1e5,class_weight='balanced',solver='liblinear').fit(x_train_matrix, train_data['Classifier'].values))])
stratified_k_fold()
#ROC_Curve()

from sklearn.externals import joblib
joblib.dump(pipeline, "Classifier5.pkl", compress=9)

[[28  7]
 [ 5 21]]
[[32  3]
 [ 0 26]]
[[34  1]
 [ 4 22]]
[[32  3]
 [ 3 23]]
[[32  3]
 [ 0 26]]
[[34  1]
 [ 3 23]]
[[35  0]
 [21  5]]
[[35  0]
 [17  8]]
[[34  1]
 [ 3 22]]
[[34  1]
 [ 9 16]]
StratifiedKFold Cross Validation Score - Using 10 folds
0.859808743169


['Classifier5.pkl']

### Gaussian NB Cross Validation Module

In [24]:
 # Perform classification with GaussianNB
from sklearn.base import TransformerMixin
class DenseTransformer(TransformerMixin):
    def transform(self, X, y=None, **fit_params):
        return X.todense()
    def fit_transform(self, X, y=None, **fit_params):
        self.fit(X, y, **fit_params)
        return self.transform(X)
    def fit(self, X, y=None, **fit_params):
        return self
    
from sklearn.naive_bayes import GaussianNB
pipeline = Pipeline([
  ('count_vectorizer',   CountVectorizer(ngram_range=(1, 2))),
    ('to_dense', DenseTransformer()),
  ('classifier', GaussianNB())])
stratified_k_fold()
#ROC_Curve()

from sklearn.externals import joblib
joblib.dump(pipeline, "Classifier6.pkl", compress=9)

[[23 12]
 [ 1 25]]
[[29  6]
 [ 1 25]]
[[20 15]
 [ 0 26]]
[[21 14]
 [ 1 25]]
[[20 15]
 [ 0 26]]
[[22 13]
 [ 0 26]]
[[29  6]
 [14 12]]
[[28  7]
 [10 15]]
[[28  7]
 [ 7 18]]
[[25 10]
 [ 9 16]]
StratifiedKFold Cross Validation Score - Using 10 folds
0.756010928962


['Classifier6.pkl']